In [ ]:
# Working Code
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import base64, time
# from PIL import Image
from io import BytesIO
from anticaptchaofficial.imagecaptcha import *
import os




from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless=new")

def aadhaar(uan):
    try:
        aadhaar_no=int(uan)
    except:
        message={'message':'Data not provided properly.'}
        return message
    if len(str(uan))!=12:
        return ('Invalid aadhaar card number!')
    else:
        for i in range(0,3):
#             browser = webdriver.Edge(options=chrome_options)
            browser = webdriver.Chrome(options=chrome_options)
            browser.get('https://myaadhaar.uidai.gov.in/verifyAadhaar')
#             browser.implicitly_wait(90)
            uid = browser.find_element('xpath','//*[@id="root"]/div/div[3]/div/div/div[1]/div/form/div/div[1]/div/div/div/input')
            uid.send_keys(str(uan))
            print("aadhaar input done")
            time.sleep(3)
            html = browser.page_source
            sub1 = "image;base64,"
            sub2 = '"'

            base64_image = ''.join(html.split(sub1)[1].split(sub2)[0])

            image_bytes = base64.b64decode(base64_image)
            
            image = Image.open(BytesIO(image_bytes))
            
            image.save(str(uan)+'.png')

            solver = imagecaptcha()
            solver.set_verbose(1)
            ANTI_KEY = '5d6fae2a97ec0efcf655040e8b17010b'
            solver.set_key(ANTI_KEY)
            captcha_text = solver.solve_and_return_solution(str(uan)+'.png')

            captcha = browser.find_element('xpath','//*[@id="root"]/div/div[3]/div/div/div[1]/div/form/div/div[2]/div[1]/div/div/div/input')
            captcha.send_keys(captcha_text)
            print("capcha also done")
            time.sleep(2)
            browser.find_element('xpath','//*[@id="root"]/div/div[3]/div/div/div[1]/div/form/div/div[3]/div/button').click()

            time.sleep(5)
            try:
                result={
                    'age':''.join(browser.page_source.split('Age Band</span><span>')[1].split('</span></div><div')[0]),
                    'gender':''.join(browser.page_source.split('Gender</span><span>')[1].split('</span></div><div')[0]),
                    'state':''.join(browser.page_source.split('State</span><span>')[1].split('</span></div><div')[0]),
                    'mobile':''.join(browser.page_source.split('Mobile</span><span>')[1].split('</span></div></div><div')[0]),
                }
                browser.close()
                file_path=(str(uan)+'.png')
                if os.path.exists(file_path):
                    # delete the file
                    os.remove(file_path)
                    print(f"{file_path} has been deleted.")
                else:
                    # raise an error if the file does not exist
                    raise ValueError(f"{file_path} does not exist.")
                return result
            except:
                result={'message':'No. result found! Please check the aadhaar card number or try again later.'}
                if i ==2:
                    return result
                else:
                    pass
aadhaar(802522527780)